In [1]:
import tensorflow as tf
import numpy as np
import kagglehub
from tensorflow.keras.layers import Conv2D , MaxPooling2D , Dropout , Flatten , Dense
from tensorflow.keras.models import Sequential

In [2]:
path = kagglehub.dataset_download('omkargurav/face-mask-dataset')
print(f'path to files :{path}')

path to files :/kaggle/input/face-mask-dataset


In [3]:
dataset = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/face-mask-dataset/data',
    labels = 'inferred',
    image_size = (128 , 128),
    label_mode = 'binary',
    batch_size = 32,
    shuffle = True ,
    seed = 123
)

Found 7553 files belonging to 2 classes.


In [4]:
dataset_size = tf.data.experimental.cardinality(dataset).numpy()
validation_size =  int(0.15*dataset_size)
train_size =  int(0.7*dataset_size)
test_size = dataset_size - train_size - validation_size


train_data = dataset.take(train_size)
remaining_data = dataset.skip(train_size)
val_data = remaining_data.take(validation_size)
test_data = remaining_data.skip(validation_size)


In [5]:


#Define normalization function
def normalize(image, label):
    """Normalizes pixel values to be between 0 and 1"""
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

#Apply normalization and optimize pipeline
train_data = train_data.map(normalize).cache().prefetch(tf.data.AUTOTUNE)
val_data = val_data.map(normalize).cache().prefetch(tf.data.AUTOTUNE)
test_data = test_data.map(normalize).cache().prefetch(tf.data.AUTOTUNE)

In [6]:
cnn_normal_model = Sequential([
    Conv2D(16  , (5,5) , activation='relu' , input_shape = (128 , 128 , 3)) ,
    MaxPooling2D((2,2)) ,
    Conv2D(32  , (3,3) , activation='relu') ,
    MaxPooling2D((2,2)) ,
    Conv2D(32  , (3,3) , activation='relu') ,

    # flattening
    Flatten() ,

    #Dense layers
    Dense(128 , activation='relu'),
    Dropout(0.5),
    Dense(64 , activation ='relu'),
    Dropout(0.3),
    Dense (1 , activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
optimizer = tf.optimizers.Adam(learning_rate=0.001)

cnn_normal_model.compile (
    optimizer = optimizer ,
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [8]:
cnn_normal_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 124, 124, 16)   │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 62, 62, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 60, 60, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,234,817 (12.34 MB)

 Trainable params: 3,234,817 (12.34 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
from tensorflow.keras.callbacks import EarlyStopping

call_backs = [EarlyStopping( patience=3, monitor='val_loss', restore_best_weights=True)]

history = cnn_normal_model.fit(
    train_data ,
    validation_data  = val_data ,
    epochs = 5 ,
    callbacks = call_backs
)

Epoch 1/5
165/165 ━━━━━━━━━━━━━━━━━━━━ 30s 132ms/step - accuracy: 0.7164 - loss: 0.5477 - val_accuracy: 0.8679 - val_loss: 0.2926
Epoch 2/5
165/165 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.8782 - loss: 0.3056 - val_accuracy: 0.9080 - val_loss: 0.2347
Epoch 3/5
165/165 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9108 - loss: 0.2432 - val_accuracy: 0.9062 - val_loss: 0.2225
Epoch 4/5
165/165 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9269 - loss: 0.2067 - val_accuracy: 0.9214 - val_loss: 0.2138
Epoch 5/5
165/165 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9355 - loss: 0.1704 - val_accuracy: 0.9348 - val_loss: 0.1917


In [10]:
test_loss , test_acc = cnn_normal_model.evaluate(test_data)
print(f'test loss : {test_loss}')
print(f'test Accuarcy : {test_acc}')

37/37 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - accuracy: 0.9426 - loss: 0.1736
test loss : 0.16483378410339355
test Accuarcy : 0.940156102180481


In [11]:
cnn_normal_model.save('face detector 2nd edition.keras')